In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences  
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from soynlp.normalizer import *

In [ ]:
train_data = pd.read_csv("train_data.csv", sep="|", quoting=3, encoding="utf-8")
test_data = pd.read_csv("test_data.csv", sep="|", quoting=3, encoding="utf-8")

In [ ]:
print("train_data의 길이 :", len(train_data))
print("test_data의 길이 :", len(test_data))

In [ ]:
# 불용어 정의
stopwords = ["아", "아이구", "아이쿠", "아이고", "어", "을", "으로", "로", "에게", "입각하여", "기준으로", "저", "소인", "소생", "저희", "지말고", "하지마", "하지마라", "물론", "막론하고", "관계없이", "그런데", "든간에", "설사", "비록", "더라도", "만 못하다", "불문하고", "향하여", "향해서", "향하다", "쪽으로", "틈타", "타다", "오르다", "제외하고", "이 외에", "이 밖에", "하여야", "비로소", "외에도", "이곳", "부터", "기점으로", "따라서", "할 생각이다", "그렇게 함으로써",  "일때", "할때", "앞에서", "중에서", "보는데서", "으로써", "반드시", "할줄알다",  "할수있다", "할수있어", "임에 틀림없다", "한다면", "등", "등등", "제", "겨우", "다만", "할뿐", "딩동", "댕그", "대해서", "대하면", "훨씬", "얼마나", "얼마만큼", "얼마큼", "남짓", "여", "얼마간", "약간", "다소", "조금", "다수",
"얼마", "지만", "또한", "그러나", "하지만", "이외에도", "대해 말하자면", "뿐이다", "반대로 말하자면", "이와 반대로","바꾸어서 말하면", "바꾸어서 한다면",  "그렇지않으면", "까악", "비걱거리다", "응당", "해야한다","에 가서", "각", "각각", "각종", "각자", "제각기", "하도록하다", "고로", "한 까닭에", "하기 때문에", "거니와", "이지만", "대하여", "관하여", "관한", "과연", "실로", "생각한대로", "진짜로", "한적이있다", "하곤하였다", "하", "하하", "허허", "거바", "와", "오", "왜", "어째서", "하겠는가", "어느곳", "더군다나", "더욱이는", "어느때", "언제", "야", "이봐", "어이", "여보시오", "흥", "휴", "헉헉", "헐떡헐떡", "영차", "여차", "어기여차", "끙끙", "앗", "콸콸", "졸졸", "좍좍", "뚝뚝", "주룩주룩", "솨", "우르르", "그래도", "또", "그리고",  "바꾸어말하면",
"바꾸어말하자면", "혹은", "혹시", "답다", "및", "그에 따르는", "설령", "가령", "하더라도", "일지라도", "몇", "거의", "하마터면", "인젠", "이젠", "된바에야", "된이상", "그위에", "점에서 보아", "비추어 보아", "고려하면", "하게될것이다", "일것이다", "비교적", "보다더", "비하면", "시키다", "하게하다", "할만하다", "의해서", "연이서", "이어서", "잇따라", "뒤따라", "뒤이어", "결국", "의지하여", "기대여","통하여", "자마자", "더욱더", "불구하고", "얼마든지", "마음대로", "주저하지 않고", "곧", "즉시", "바로", "당장", "하자마자", "밖에 안된다", "하면된다", "그렇지","요컨대", "다시 말하자면", "바꿔 말하면", "즉", "구체적으로", "말하자면", "시작하여", "시초에","이상", "허", "헉", "허걱", "바와같이", "해도좋다", "해도된다","더구나", "하물며", "와르르", "팍", "퍽", "펄렁",
"이래",  "하고있었다", "이었다","에서", "로부터", "까지", "예하면", "해요", "함께", "같이", "더불어", "마저", "마저도", "양자", "모두", "습니다", "가까스로", "하려고하다", "즈음하여", "다른", "다른 방면으로", "해봐요", "습니까", "말할것도 없고", "무릎쓰고", "개의치않고", "하는것만 못하다", "하는것이 낫다", "매", "매번", "모", "로써", "갖고말하자면", "어디", "어느쪽", "어느해", "어느 년도", "라 해도", "언젠가", "저기", "저쪽", "저것", "그럼", "그러면", "요만한걸", "그래", "저것만큼", "그저", "할 줄 안다", "할 힘이 있다",  "당신", "어찌", "설마", "할지라도", "할망정", "구토하다", "게우다", "토하다", "메쓰겁다", "옆사람", "퉤", "쳇", "의거하여", "근거하여", "의해", "따라", "힘입어", "다음", "버금", "두번째로", "기타", "첫번째로", "나머지는", "그중에서", "견지에서",
"형식으로 쓰여", "입장에서", "위해서", "단지", "의해되다", "하도록시키다", "뿐만아니라", "반대로", "전후", "전자", "앞의것", "잠시", "잠깐", "하면서", "그렇지만", "다음에", "그러한즉","그런즉", "남들", "아무거나", "어찌하든지", "같다", "비슷하다", "예컨대", "이럴정도로", "만약", "만일", "위에서 서술한바와같이", "인 듯하다", "하지 않는다면", "만약에", "무엇", "무슨", "어느", "어떤", "아래윗", "조차", "한데", "그럼에도 불구하고", "여전히", "심지어", "까지도", "조차도", "않기 위하여", "때", "시각", "무렵","시간", "동안", "하여금", "예", "누구",  "누가 알겠는가", "아무도", "줄은모른다", "줄은 몰랏다", "하는 김에", "겸사겸사", "하는바", "한 이유는", "그러니", "때문에", "그", "너희", "그들", "너희들", "타인","것", "것들", "너", "위하여", "공동으로", "동시에",
"하기 위하여",  "어찌하여","무엇때문에", "붕붕", "엉엉", "휘익", "오호", "하는 편이 낫다","흐흐", "상대적으로 말하자면", "마치", "아니라면", "쉿", "그렇지 않으면", "그렇지 않다면", "안 그러면", "아니었다면", "하든지", "아니면", "이라면", "좋아", "알았어", "하는것도", "어쩔수 없다", "하나", "일", "일반적으로", "일단", "한켠으로는", "오자마자", "이와같다면", "전부", "한마디", "한항목", "근거로", "하기에", "아울러", "하지 않도록", "않기 위해서", "이르기까지", "이 되다", "로 인하여", "까닭으로", "이유만으로", "이로 인하여", "그래서", "이 때문에", "그러므로", "알 수 있다", "결론을 낼 수 있다", "으로 인하여", "있다", "어떤것", "관계가 있다", "관련이 있다", "연관되다", "에 대해", "이리하여", "그리하여", "여부", "하기보다는", "하느니", "하면 할수록", "이러이러하다",
"하구나",  "하도다","다시말하면",  "다음으로", "에 있다", "에 달려 있다", "우리들", "하기는한데", "어떻게", "어떻해", "어찌됏어", "자", "이", "이쪽", "여기", "이번", "이렇게말하자면", "이러한", "이와 같은", "요만큼", "얼마 안 되는 것", "이만큼", "이 정도의", "이렇게 많은 것", "이와 같다", "이때", "이렇구나", "것과 같이", "와 같은 사람들", "부류의 사람들", "중의하나", "에 한하다", "하기만 하면", "도착하다", "까지 미치다", "도달하다", "정도에 이르다", "할 지경이다", "결과에 이르다","관해서는",  "여러분", "하고 있다", "한 후", "자기","자기집", "자신", "우에 종합한것과같이", "총적으로 보면", "총적으로 말하면", "총적으로", "대로 하다", "으로서", "그만이다", "할 따름이다", "봐", "봐라", "아이야", "아니", "와아", "응", "아이","하다",  "게", "분들", "는", "가", "인",
"좀", "도", "에", "고","지", "임", "네", "이다", "은", "들","한", "나", "다", "의", "걍", "님", "요", "를"]

In [ ]:
print("train_data의 길이 :", len(train_data))
print("test_data의 길이 :", len(test_data))

In [ ]:
okt = Okt()

X_train = []
for sentence in tqdm(train_data['reviews']):
    # 토큰화
    tokenized_sentence = okt.morphs(sentence, stem=True)
    
    # 불용어 제거
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords]
    X_train.append(stopwords_removed_sentence)

In [ ]:
X_test = []
# 데이터프레임 합친 것
for sentence in tqdm(test_data['reviews']):
    # 토큰화
    tokenized_sentence = okt.morphs(sentence, stem=True)
    
    # 불용어 제거
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords]
    X_test.append(stopwords_removed_sentence)

In [ ]:
    # 한글에서 자음 또는 모음만으로 이루어진 문자 삭제
    # stopwords_removed_sentence = [word for word in stopwords_removed_sentence if not re.match(r'^[ㄱ-ㅎㅏ-ㅣ]+$', word)]
    

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

print(tokenizer.word_index)

In [ ]:
print(X_train[:3])

In [ ]:
threshold = 3

# 단어의 수
total_cnt = len(tokenizer.word_index)

# 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
rare_cnt = 0

# 훈련 데이터의 전체 단어 빈도수 총 합
total_freq = 0

# 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합
rare_freq = 0

# 단어와 빈도수의 쌍(pair)을 key와 value로 받기
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

In [ ]:
# 전체 단어 개수 중 빈도수 2이하인 단어는 제거.
# 0번 패딩 토큰을 고려하여 + 1
vocab_size = total_cnt - rare_cnt + 1
print('단어 집합의 크기 :',vocab_size)

In [ ]:
# 단어 집합을 케라스 토크나이저의 인자로 넘겨주고
# 텍스트 시퀀스를 정수 시퀀스로 변환
tokenizer = Tokenizer(vocab_size) 
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
print(X_train[:3])

In [ ]:
y_train = np.array(train_data['star'])
y_test = np.array(test_data['star'])

In [ ]:
# 빈도수 2이하인 단어를 제거하고 생긴 빈 샘플을 제거하기 위해
# 길이가 0인 샘플들의 인덱스를 받아옴
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]

In [ ]:
# 빈 샘플들을 제거
# X_train = np.delete(X_train, drop_train, axis=0)
y_train = np.delete(y_train, drop_train, axis=0)
print(len(X_train))
print(len(y_train))

서로 다른 길이의 샘플들의 길이를 동일하게 맞춰주는 패딩 작업

In [ ]:
print('리뷰의 최대 길이 :',max(len(review) for review in X_train))
print('리뷰의 평균 길이 :',sum(map(len, X_train))/len(X_train))
plt.hist([len(review) for review in X_train], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

가장 긴 리뷰의 길이는 69이며, 그래프를 봤을 때 전체 데이터의 길이 분포는 대체적으로 약 11내외의 길이를 가지는 것을 볼 수 있습니다. 모델이 처리할 수 있도록 X_train과 X_test의 모든 샘플의 길이를 특정 길이로 동일하게 맞춰줄 필요가 있습니다. 특정 길이 변수를 max_len으로 정합니다. 대부분의 리뷰가 내용이 잘리지 않도록 할 수 있는 최적의 max_len의 값은 몇일까요? 전체 샘플 중 길이가 max_len 이하인 샘플의 비율이 몇 %인지 확인하는 함수를 만듭니다.


In [ ]:
# 전체 샘플 중 길이가 max_len 이하인 샘플의 비율이 몇 %인지 확인하는 함수
def below_threshold_len(max_len, nested_list):
  count = 0
  for sentence in nested_list:
    if(len(sentence) <= max_len):
        count = count + 1
  print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (count / len(nested_list))*100))


In [ ]:
max_len = 70
below_threshold_len(max_len, X_train)

In [ ]:
# 하이퍼파라미터인 임베딩 벡터의 차원 크기
embedding_dim = 100
# 은닉 상태의 크기
hidden_units = 128
# 모델 학습

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.2)

In [ ]:
# 테스트 모델 저장
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

In [ ]:
with open('tokenizer.pickle', 'wb') as handle:
     pickle.dump(tokenizer, handle)

with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [ ]:
def sentiment_predict(new_sentence):
  new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
  new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
  new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
  encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
  pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
  score = float(loaded_model.predict(pad_new)) # 예측
  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))